
# Taller 1 - Fase 3: Aplicación de la Ingeniería de Prompts (EcoMarket)

Este notebook desarrolla la **Fase 3 del Taller IAGEN**, aplicando **ingeniería de prompts** 
para el caso de estudio **EcoMarket** (optimización del servicio al cliente en e-commerce).  

Se usará un **dataset simulado de pedidos (`pedidos.xlsx`)** y la **API de OpenAI** para probar:
- Consultas de **estado de pedido**  
- Consultas de **devolución de producto**  


In [ ]:

!pip install openai pandas


In [ ]:

import pandas as pd
from openai import OpenAI

# Cargar dataset de pedidos simulado
df = pd.read_excel("pedidos.xlsx")
df


In [ ]:

system_prompt = """
Eres un asesor experto en atención al cliente de EcoMarket.  
Debes responder con empatía, claridad y precisión.  
Consulta siempre la base de datos interna de pedidos o políticas de devoluciones.  
"""

def build_prompt_estado_pedido(tracking_number, pedido):
    return f"""
Actúa como un agente de atención al cliente de EcoMarket.  
El cliente solicita el estado del pedido con número de seguimiento {tracking_number}.  

Información encontrada en la base de datos:  
- Estado: {pedido['estado']}  
- Fecha estimada de entrega: {pedido['fecha_entrega_estimada']}  
- Enlace de rastreo: {pedido['link_rastreo']}  
- Comentario: {pedido['comentario']}  

Genera una respuesta amable y clara para el cliente.  
Si el pedido está retrasado, pide disculpas y explica el motivo.  
"""

def build_prompt_devolucion(producto, elegible):
    if elegible:
        return f"""
El cliente desea devolver el producto '{producto}'.  
Este producto **sí es elegible** para devolución según las políticas de EcoMarket.  

Guía al cliente con pasos claros para iniciar la devolución (plazos, pasos y condiciones).  
Responde siempre con empatía y claridad.  
"""
    else:
        return f"""
El cliente desea devolver el producto '{producto}'.  
Este producto **NO es elegible** para devolución (ej. productos de higiene o perecederos).  

Responde con empatía y explica claramente la razón, ofreciendo alternativas si existen.  
"""


In [ ]:

# Configura tu API Key antes de usar
client = OpenAI(api_key="TU_API_KEY")

def consultar_estado_pedido(tracking_number):
    pedido = df[df["tracking_number"] == tracking_number]
    if pedido.empty:
        return f"No se encontró el pedido con número {tracking_number}."
    pedido_info = pedido.iloc[0]
    user_prompt = build_prompt_estado_pedido(tracking_number, pedido_info)
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

def procesar_devolucion(producto):
    # Definir criterios de elegibilidad
    no_elegibles = ["Shampoo sólido", "Jabón artesanal"]
    elegible = producto not in no_elegibles
    
    user_prompt = build_prompt_devolucion(producto, elegible)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content


In [ ]:

# Ejemplo: Estado de pedido
print(consultar_estado_pedido(23456))

# Ejemplo: Devolución de producto
print(procesar_devolucion("Cepillo de bambú"))
print(procesar_devolucion("Shampoo sólido"))



## ✅ Conclusiones

- La **ingeniería de prompts** permite personalizar las respuestas de IA para casos reales de negocio.  
- En EcoMarket, se logra **reducir el tiempo de respuesta** y **mejorar la empatía** en la comunicación.  
- Los prompts ajustados al contexto (pedido vs devolución) aseguran que el cliente reciba respuestas útiles.  

Este notebook puede integrarse en un flujo más amplio (ej. API o chatbot) para automatizar la atención al cliente.  
